In [7]:
import requests
import pandas as pd
from tqdm import tqdm


from datetime import datetime
import logging
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%Y%m%d%H%M%S")
logging.basicConfig(filename=f'scrapping_log/qatar_scrapping{timestampStr}.log',  level=logging.DEBUG,
    format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
timestampStr
logging.info(f'INITIALISING ..... {timestampStr} .....')

FOLDER = 'dataset'

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import accuracy_score


import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.pipeline import Pipeline

import joblib

import predictgame as pg
import fifa as fifa
import odds as odds
from worldcuptransformers import CustomTransformer

import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)


from datetime import datetime
import logging
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%Y%m%d%H%M%S")
logging.basicConfig(filename=f'compute_wc_log/compute-{timestampStr}.log',  level=logging.DEBUG,
    format='%(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
timestampStr

'20221104185812'

In [8]:
fifa = fifa.Fifa()
fifa.get_last_rank('Georgia')

78

In [9]:
fifa.get_avg_rank(team='Georgia', since='2021-01-01')

87.3076923076923

In [13]:
o = odds.Bets()
o.get_odds('Iraq','Brazil')

(1.3036642195767196, 2.081262896825397, 3.1041304894179893)

In [ ]:

!pip install beautifulsoup4

In [ ]:
import bs4 as BeautifulSoup
from bs4 import BeautifulSoup

In [ ]:
wcs= ['https://www.betexplorer.com/soccer/world/world-cup-2022/']
group_stage = []

# https://www.betexplorer.com/soccer/world/friendly-international-2013/results/?month=2013-02
urls = []
games_bets_rows = []
for wc in tqdm(wcs):
    url = wc
    print(wc)
    #try:
    page = requests.get(url)

    content = page.content


    soup = BeautifulSoup(page.content, 'html.parser')


    tr_rows = soup.find_all('tr')
    #logging.info(f'LEN TR_ROWS = {len(tr_rows)}')

    ix = 0
    for row in tr_rows:
        #print(ix)
        ix=ix+1
        r = str(row)
        #print(r)

        if 'table-main__daysign' in r:
            logging.info(r)
            matchday = r.split('/soccer/world/world-cup-2022')[1]

            temp = BeautifulSoup(r, 'html.parser')
            spans = temp.find_all('span')
            i=0
            for span in spans:
                if i < 2:

                    s = str(span)
                    s = s.replace('<','').replace('>','').replace('strong','').replace('span','').replace('/','')
                    #logging.info(f'{s}')
                    if i == 0:
                        home_team=s
                    else:
                        away_team=s
                    i=i+1


            odds = temp.find_all('td', class_='table-main__odds')
            input_tag = temp.find("td", {"name":"City"})
            odds_list=[]
            valid_odds=False
            for odd in odds:
                o = str(odd).split('data-odd=')
                #logging.info(o[1].split('"')[1])
                odds_list.append(o[1].split('"')[1])
                valid_odds=True
                
                
            logging.info(f'{home_team} {away_team} {odds_list}')
            group_stage.append([home_team,away_team,odds_list[0],odds_list[1],odds_list[2]])
                
"""
            games = temp.find_all('td', class_='h-text-center')
            
            
            for game in games:
                gam= str(game)
                #logging.info(gam)
                # get the game + key
                g = gam.split('href="')[1].split('">')[0]
                g = g.split('/')
                g = f"{g[1]}/{g[2]}/{g[3]}/{g[4]}-{g[5]}"
                # get game score
                s = gam.split('</a')[0]
                s = s.split('/">')[1]
                s = s.split(' ')[0]
                if ':' in s:
                    logging.info(s)
                    correct_score=s
                    game_entry=g
                    game_url = url
                    urls.append([g,url,s])
            logging.info(matchday.split('<')[0])
            matchday = matchday.split('<')[0]

            if valid_odds:
                games_bets_rows.append([game_entry, matchday, home_team, away_team, correct_score, odds_list[0], odds_list[1], odds_list[2], game_url ])
"""

    #except:
    #    logging.info('no-game')
    #    print('no Game')

        
df = pd.DataFrame(games_bets_rows)
df.to_csv(f'data/urls_{timestampStr}.csv', index=False)
df.to_csv(f'oddsportal_urls_{timestampStr}.csv')
df.drop_duplicates()

In [ ]:
group_stage

In [ ]:
games=pd.DataFrame(group_stage)
games.columns = ['home_team','away_team','1N2_1','1N2_N','1N2_2']
games['1N2_1']=pd.to_numeric(games['1N2_1'])
games['1N2_N']=pd.to_numeric(games['1N2_N'])
games['1N2_2']=pd.to_numeric(games['1N2_2'])
games.dtypes

In [ ]:
games

In [ ]:
stage = []
for idx, game in tqdm(games.iterrows()):
    #print(game.home_team, game.away_team)
    home_fifa = fifa.get_last_rank(game.home_team)
    away_fifa = fifa.get_last_rank(game.away_team)
    payload=[game['1N2_1'],game['1N2_N'],game['1N2_2'],home_fifa,away_fifa,1,'2022-11-29',game.home_team,game.away_team,'-:-']
    data ={}
    data['game1']=payload
    play=pd.DataFrame.from_dict(data).T
    play.columns = ['1N2_1','1N2_N','1N2_2','home_team_rank_FIFA','away_team_rank_FIFA','season','matchday','home_team','away_team','correct_score']
    #print(play)

    g = pg.PredictGame(play)
    p = g.predict()
    t=p[0][0][7]
    #print(f'Prediction= {p}')

    result_proba_1N2=[]
    result_proba_12=[]
    rem = f'GAME {0+1} - {play.home_team.values[0]} Vs. {play.away_team.values[0]} - Prediction={p[0][0][4]}/{p[0][0][5]} = {np.int(p[0][0][8][0])}:{np.int(p[0][0][8][1])}'
    stage.append([game.home_team,game.away_team,np.int(p[0][0][8][0]),np.int(p[0][0][8][1])])

In [ ]:
stage

In [ ]:
pred=pd.DataFrame(stage)
pred.columns = ['home_team','away_team','home_score','away_score']
for idx, r in pred.iterrows():
    if r.home_score > r.away_score:
        pred.loc[idx,'h_points']=3
        pred.loc[idx,'a_points']=0
    elif r.home_score == r.away_score:
        pred.loc[idx,'h_points']=1
        pred.loc[idx,'a_points']=1
    else:
        pred.loc[idx,'h_points']=0
        pred.loc[idx,'a_points']=3

pred

# 1st Stage Result and table

In [ ]:
# World Cup Table
#The structure of the world cup is stored in a json file
WC_TABLE = 'wc_2022_table.json'
import pandas as pd
import json
import os


# Reading the json as a dict
with open('dataset/wc_2022_table.json') as json_data:
    data = json.load(json_data)

wc_table = pd.DataFrame.from_dict(data['stage'], orient='index').T.reset_index()
#print(wc_table)
table = pd.DataFrame()
for idx, r in wc_table.iterrows():
    teams = r.groups
    group = r['index']
    for team in teams:
        # look in predict table for point and score
        h=pred.query('home_team == @team')[['home_score','away_score','h_points']]
        h.columns=['for','against','points']
        a=pred.query('away_team == @team')[['away_score','home_score','a_points']]
        a.columns=['for','against','points']

        frames = [a,h]
        t=pd.DataFrame(pd.concat(frames).sum()).T
        t['team']=team      
        t['group']=group

        frames = [table, t]
        table = pd.concat(frames)  

In [ ]:
cols = ['group','team','for','against','diff','points']
table['diff']= table['for']-table['against']
table=table[cols].sort_values(['group','points','diff','for','against'], ascending=[True,False,False,False,True])
logging.info('')
logging.info('Tables after Group Stage')
logging.info('---------------------------------------')
logging.info(table)

In [ ]:
first_in_group = table.groupby(['group']).nth(0).copy()
for idx, r in first_in_group.iterrows():
    grp = idx.split('-')
    code = f"1{grp[1].upper()}"
    first_in_group.loc[idx,'code']=code


second_in_group = table.groupby(['group']).nth(1).copy()
for idx, r in second_in_group.iterrows():
    grp = idx.split('-')
    code = f"2{grp[1].upper()}"
    second_in_group.loc[idx,'code']=code




frames = [first_in_group,second_in_group]
logging.info('')
logging.info('1st After Group Stage')
logging.info('---------------------------------------')
logging.info(first_in_group)

logging.info('')
logging.info('2nd After Group Stage')
logging.info('---------------------------------------')
logging.info(second_in_group)

In [ ]:
first_in_group

In [ ]:
second_in_group

# Knockout Stage

In [ ]:
logging.info('-----------------------------------------------------------------------------')
logging.info('    KNOCKOUT STAGE')
logging.info('')

In [ ]:
next_game = pd.concat(frames)
next_game
ko16 = pd.DataFrame.from_dict(data['knockouts'], orient='index').T.reset_index()
ko16
games16 = []
for idx, r in ko16.iterrows():
    g1 = r['knockout-16'][0]
    g2 = r['knockout-16'][1]

    t1 = g1[0]
    t2 = g1[1]

    games16.append([next_game.query('code == @t1').team.values[0],next_game.query('code == @t2').team.values[0]])

    t1 = g2[0]
    t2 = g2[1]

    games16.append([next_game.query('code == @t1').team.values[0],next_game.query('code == @t2').team.values[0]])


logging.info('******* ROUND 16')
logging.info(games16)
games16


In [ ]:
# all time games
alg = pd.read_csv(f'{FOLDER}/training_dataset.csv')
alg.shape
# if no data are found so this function will create a dummy entries with teams
# the scope of data is based on feat which is the collection of data need

def define_unknown_game(home, away):
    """_summary_
    Look inside the training_dataset for both teams games and fifa ranking and the collect
    same pattern game

    Args:
        home (_type_): _description_
        away (_type_): _description_
    """
    #print(home, away)
    # get a list of ranking FIFA
    rk = pd.read_csv('dataset/fifa_ranking-2021-05-27.csv')
    rk['rank_date']=pd.to_datetime(rk['rank_date'])
    rk['rank_year']=rk.rank_date.dt.year
    rk = rk.query('rank_year == 2018').copy()
    rk = rk.groupby('country_full').mean()

    ts = pd.read_csv('dataset/training_dataset.csv')
    ts['FIFA_diff']=abs(ts['home_team_rank_FIFA']-ts['away_team_rank_FIFA'])

    rk= rk.query('country_full == @home or country_full == @away')
    try:
        home_rank = rk.query('country_full == @home')['rank'][0]
        away_rank = rk.query('country_full == @away')['rank'][0]
    except:
        home_rank=99
        away_rank=99

    diff = abs(home_rank - away_rank)

    subset_ts = ts.query('(FIFA_diff > @diff*0.85 and FIFA_diff < @diff*1.15)')
    subset_ts= pd.DataFrame(subset_ts.mean()).T
    subset_ts['matchday']=None
    subset_ts['home_team']=home
    subset_ts['away_team']=away
    subset_ts['correct_score']='0:0'
    try:
        return rk.query('country_full == @home')['rank'][0],rk.query('country_full == @away')['rank'][0], subset_ts
    except:
        return 99,99, subset_ts



rh, ra, ts = define_unknown_game('Belgium','Senegal')
rh, ra


In [ ]:
for game in games16:
    rh = fifa.get_last_rank(game[0])
    ra = fifa.get_last_rank(game[1])
    print(game[0], game[1], np.round(rh,2), np.round(ra,2))